## Домашняя работа по теме "Embedding word2vec fasttext"

Загрузим архив с новостями

In [1]:
from corus import load_lenta
path = 'lenta-ru-news.csv.gz'
records = load_lenta(path)

In [2]:
next(records)

LentaRecord(
    url='https://lenta.ru/news/2018/12/14/cancer/',
    title='Названы регионы России с\xa0самой высокой смертностью от\xa0рака',
    text='Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости. По словам Голиковой, чаще всего онкологические заболевания становились причиной смерти в Псковской, Тверской, Тульской и Орловской областях, а также в Севастополе. Вице-премьер напомнила, что главные факторы смертности в России — рак и болезни системы кровообращения. В начале года стало известно, что смертность от онкологических заболеваний среди россиян снизилась впервые за три года. По данным Росстата, в 2017 году от рака умерли 289 тысяч человек. Это на 3,5 процента меньше, чем годом ранее.',
    topic='Россия',
    tags='Общество',
    date=None
)

Импортируем нужные библиотеки

In [16]:
import pandas as pd
import numpy as np

from gensim.models import Word2Vec, FastText
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import string
import annoy
import re
from tqdm import tqdm
from tqdm import tqdm_notebook
import nltk
import mmap
from nltk.tokenize import word_tokenize
from razdel import tokenize

tqdm.pandas()
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [6]:
corpus_title = []
corpus_text = []

for num, iter in tqdm(enumerate(records)):
    corpus_title.append(iter.title)
    corpus_text.append(iter.text)
    if num == 99999:
        break

99999it [00:10, 9768.32it/s] 


In [7]:
corpus_title[70], corpus_text[70]

('Обнаружен способ взломать любую учетную запись Microsoft',
 'Исследователь в области кибербезопасности из Индии Сахад Нк (Sahad Nk) обнаружил уязвимости, которые позволяли злоумышленникам взломать любую учетную запись Microsoft. Подробный отчет о найденных ошибках опубликован на сайте компании-заказчика SafetyDetective. Хакер решил не подбирать пароли и не атаковать аккаунты, а зайти со стороны компании-поставщика услуг. Нк сумел в семь шагов взломать поддомен success.office.com, принадлежащий Microsoft. Оказалось, что он был некорректно сконфигурирован. Взятая под контроль часть домена участвует в процессе авторизации программ Microsoft — Outlook, Office и других. Для получения доступа к чужой учетной записи Нк послал жертве фишинговое письмо. Спам-фильтры не отсеяли бы послание, отправленное с адреса Microsoft. После перехода по ссылке, выданной «официальной» (взломанной) структурой, злоумышленник получил бы специальный токен для входа в аккаунт. Ему не понадобились бы никакие допо

In [8]:
df = pd.DataFrame(corpus_title, columns=['title'])
df['text'] = corpus_text
df.head()

,title,text
0,Австрия не представила доказательств вины росс...,Австрийские правоохранительные органы не предс...
1,Обнаружено самое счастливое место на планете,Сотрудники социальной сети Instagram проанализ...
2,В США раскрыли сумму расходов на расследование...,С начала расследования российского вмешательст...
3,Хакеры рассказали о планах Великобритании зами...,Хакерская группировка Anonymous опубликовала н...
4,Архиепископ канонической УПЦ отказался прийти ...,Архиепископ канонической Украинской православн...


Функция для очистки текста

In [9]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    p_html = re.compile(r'<.*?>')
    text = p_html.sub('', text)
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    return text

cache = {}

Функция для лемматизации текста

In [10]:
def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

Загрузка русских стоп-слов

In [11]:
stopword_ru = set(get_stop_words("ru"))
len(stopword_ru)

421

In [13]:
morph = MorphAnalyzer()

Функция препроцессинга

In [14]:
def preprocess_text(line):
    clean_line = clean_text(line)
    lemmatize_line = lemmatization(clean_line)
    return lemmatize_line

Посмотрим на результат очистки:

In [17]:
preprocess_text(df['text'].values[0])

['австрийский',
 'правоохранительный',
 'орган',
 'представить',
 'доказательство',
 'нарушение',
 'российский',
 'биатлонист',
 'антидопинговый',
 'правило',
 'сообщить',
 'посол',
 'россия',
 'вена',
 'дмитрий',
 'любинский',
 'итог',
 'встреча',
 'уполномоченный',
 'адвокат',
 'дипмиссия',
 'представитель',
 'прокуратура',
 'страна',
 'передавать',
 'тасс',
 'действовать',
 'презумпция',
 'невиновность',
 'какихлибо',
 'ограничение',
 'свобода',
 'передвижение',
 'команда',
 'добавить',
 'посольство',
 'международный',
 'союз',
 'биатлонист',
 'ibu',
 'применять',
 'санкция',
 'российский',
 'биатлонист',
 'продолжить',
 'выступление',
 'кубок',
 'мир',
 'полиция',
 'нагрянуть',
 'отель',
 'сборная',
 'россия',
 'хохфильцен',
 'вечером',
 'декабрь',
 'написать',
 'биатлонист',
 'александр',
 'логинов',
 'считать',
 'виновный',
 'махинация',
 'переливание',
 'кровь',
 'биатлонист',
 'антон',
 'шипулина',
 'попасть',
 'список',
 'полиция',
 'нанести',
 'отдельный',
 'визит',
 'трениро

Проведем очистку новостей

In [19]:
sentences = []
counter = 0

for line in tqdm_notebook(df.text):
    spls = preprocess_text(line)
    sentences.append(spls)
    counter += 1
    if counter > 100000:
        break

  0%|          | 0/100000 [00:00<?, ?it/s]

Модель Word2Vec

In [22]:
%%time
modelW2V = Word2Vec(sentences=sentences, size=100, window=5, min_count=1, workers=4)

CPU times: user 6min 29s, sys: 2.65 s, total: 6min 31s
Wall time: 2min 54s


Модель FastText

In [23]:
%%time
modelFT = FastText(sentences=sentences, size=100, window=5, min_count=1, workers=4)

CPU times: user 34min 11s, sys: 9.74 s, total: 34min 21s
Wall time: 10min 46s


In [24]:
w2v_index = annoy.AnnoyIndex(100 ,'angular')
ft_index = annoy.AnnoyIndex(100 ,'angular')

index_map = {}
counter = 0

for line in tqdm_notebook(df.text):
        n_w2v = 0
        n_ft = 0
        spls = line
        index_map[counter] = spls
        question = preprocess_text(spls)

        vector_w2v = np.zeros(100)
        vector_ft = np.zeros(100)
        for word in question:
            if word in modelW2V.wv:
                vector_w2v += modelW2V.wv[word]
                n_w2v += 1
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_w2v > 0:
            vector_w2v = vector_w2v / n_w2v
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        w2v_index.add_item(counter, vector_w2v)
        ft_index.add_item(counter, vector_ft)

        counter += 1
        # ограничение в  100000 строк
        if counter > 100000:
            break


w2v_index.build(10)
ft_index.build(10)

  0%|          | 0/100000 [00:00<?, ?it/s]

True

Посмотрим на результат работы модели

In [25]:
def get_response(question, index, model, index_map):
    question = preprocess_text(question)
    vector = np.zeros(100)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 5, )
    return [index_map[i] for i in answers]

In [28]:
TEXT = 'рассказать о жизни города'

In [29]:
get_response(TEXT, w2v_index, modelW2V, index_map)

['Житель Китая, отправившийся на велосипеде домой на празднование Нового года, месяц ехал не в том направлении. Об этом сообщает интернет-издание Shanghaiist. Полицейские остановили мужчину на трассе в районе города Уху (провинция Анхой). Он рассказал, что едет домой в Цицикар (северо-восточная провинция Хэйлунцзян) из города Жичжао (провинция Шаньдун на востоке страны), где работает. Стражи порядка пояснили мужчине, что он двигался в противоположном направлении. Рабочему, чтобы встретить праздник с родными, надо было преодолеть около двух тысяч километров. За месяц он оказался на 500 километров дальше от места назначения. По словам мужчины, велосипед он выбрал ради экономии, а направление спросил в самом начале у кого-то из прохожих. Полицейские купили путешественнику билет на поезд до родного города. Дата китайского Нового года определяется по лунному календарю. В 2017 году он приходится на 28 января. Праздничные мероприятия продлятся до 2 февраля включительно.',
 'Российский путешес

In [30]:
get_response(TEXT, ft_index, modelFT, index_map)

['В Центральной Италии произошло сильное землетрясение. Об этом сообщает Reuters. Эпицентр находился на глубине 10 километров примерно в 100 километрах к северу от итальянской столицы, магнитуда составила 5,4. Толчки ощущались в Риме и в городах трех регионов — Лацио, Абруццо и Марке. О жертвах и разрушениях пока не сообщается. По данным геологической службы США, эпицентр располагается всего в семи километрах от города Аматриче, серьезно пострадавшего от землетрясения 24 августа 2016 года. Тогда в Аматриче и соседних городах Аккумоли и Аркуате погибли 300 человек, тысячи домов были разрушены. Сейсмологи зафиксировали более 45 тысяч афтершоков, включая крупное землетрясение в октябре магнитудой 6,6. Тогда пострадали десятки человек.',
 'Взрыв произошел на химзаводе в испанском городе Патерна, провинция Валенсия. На данный момент известно об одном серьезно пострадавшем, еще 20 человек получили легкие ранения. Видео с места произошедшего публикует Ruptly. Как отмечается, на заводе, распол

__Вывод:__ обе модели справились достаточно хорошо и на тестовый текст подобрали релевантные новости. Однако, модель Word2Vec обучалась в 5 раз быстрее.